In [36]:
import datetime
import import_ipynb
import importlib
import Model
import pandas
import yfinance as yf

importlib.reload(Model)
from Model import train_model
from Model import build_test_data
from sklearn.tree import DecisionTreeRegressor  
from dateutil.relativedelta import relativedelta
from pandas_datareader import data as pdr



# Override yFinance with Pandas
yf.pdr_override() # <-- Here is the fix

importing Jupyter notebook from Model.ipynb
importing Jupyter notebook from Analytics.ipynb


In [40]:
def run_simulation(StockTickers = [], SimulationYears = 1, StartingFunds = 1000.00):
    
    TotalCash = StartingFunds
    
    # create a regressor object 
    model1 = DecisionTreeRegressor(random_state = 0) 
    model2 = DecisionTreeRegressor(random_state = 0) 
    model3 = DecisionTreeRegressor(random_state = 0) 
    model4 = DecisionTreeRegressor(random_state = 0) 
    model5 = DecisionTreeRegressor(random_state = 0) 
    
    # train our master model
    #for ticker in StockTickers:
    model1 = train_model (model1, StockTickers[0])
    model2 = train_model (model2, StockTickers[1])
    model3 = train_model (model3, StockTickers[2])
    model4 = train_model (model4, StockTickers[3])
    model5 = train_model (model5, StockTickers[4])
    
    #run purchase / sell simulation     
    total = simulate_market (model1, model2, model3, model4, model5, StockTickers, SimulationYears, StartingFunds)
    
    return total

In [49]:
def simulate_market (model1, model2, model3, model4, model5, StockTickers, SimulationYears, StartingFunds):
    
    # Okay, our model is already trained. Now we just gotta sick her on some data! 
    Total_Funds = StartingFunds
    
    # Test data shape should be (len(data), number of stocks * 9)
    # test_data.shape = (2607, 40)
    # Each ticker is 8 data points long.
    test_data = build_test_data(StockTickers)
    
    # Start SimulationYears * 365 days ago.
    # Max SimulationYears = 7
    i = 0
    for days in reversed(range (25, SimulationYears*365, 7)):
        Total_Funds = purchase_sell_stocks (model1, model2, model3, model4, model5, test_data, days, Total_Funds)
        i+=1
        if (i %52 == 0):
            print(Total_Funds)
    
    return Total_Funds

In [45]:
def purchase_sell_stocks(model1, model2, model3, model4, model5, test_data, days, Total_Funds):
    
    number_of_stocks_purchased = 0
    
    total_gains_1 = model1.predict(test_data[0:len(test_data) - days, 0:8])[-1] - test_data[len(test_data) - days - 5, 7]
    total_gains_2 = model2.predict(test_data[0:len(test_data) - days, 8:16])[-1] - test_data[len(test_data) - days - 5, 15]
    total_gains_3 = model3.predict(test_data[0:len(test_data) - days, 16:24])[-1] - test_data[len(test_data) - days - 5, 23]
    total_gains_4 = model4.predict(test_data[0:len(test_data) - days, 24:32])[-1] - test_data[len(test_data) - days - 5, 31]
    total_gains_5 = model5.predict(test_data[0:len(test_data) - days, 32:40])[-1] - test_data[len(test_data) - days - 5, 39]
    
    
    maximum_gains = max(total_gains_1, total_gains_2, total_gains_3, total_gains_4, total_gains_5)
    
    if (maximum_gains < 0):
        return Total_Funds
    
    elif (maximum_gains == total_gains_1):
        number_of_stocks_purchased = Total_Funds / test_data[len(test_data) - days, 7]
        Total_Funds = number_of_stocks_purchased * test_data[len(test_data) - days + 5, 7]
        
    elif (maximum_gains == total_gains_2):
        number_of_stocks_purchased = Total_Funds / test_data[len(test_data) - days, 15]
        Total_Funds = number_of_stocks_purchased * test_data[len(test_data) - days + 5, 15]       
        
    elif (maximum_gains == total_gains_3):
        number_of_stocks_purchased = Total_Funds / test_data[len(test_data) - days, 23]
        Total_Funds = number_of_stocks_purchased * test_data[len(test_data) - days + 5, 23]
        
    elif (maximum_gains == total_gains_4):
        number_of_stocks_purchased = Total_Funds / test_data[len(test_data) - days, 31]
        Total_Funds = number_of_stocks_purchased * test_data[len(test_data) - days + 5, 31]
        
    elif (maximum_gains == total_gains_5):
        number_of_stocks_purchased = Total_Funds / test_data[len(test_data) - days, 39]
        Total_Funds = number_of_stocks_purchased * test_data[len(test_data) - days + 5, 39]
    
    
    #print(maximum_gains)
    return Total_Funds